In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
os.getcwd()

'/media/jdcloud'

In [2]:
import scipy.io as sio
train_dataset_path = os.getcwd()+"/Train/"
val_dataset_path = os.getcwd()+"/Val/"
#record = "TRAIN0001"
#ecg = sio.loadmat(dataset_path+record+'.mat')
#ecg["II"][0].shape

In [3]:
train_files = os.listdir(train_dataset_path)
train_files.sort()
val_files = os.listdir(val_dataset_path)
val_files.sort()

In [4]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from utils import extract_basic_features

import wfdb
import os
import wfdb.processing as wp
import matplotlib.pyplot as plt
from scipy import signal
from utils import find_noise_features, extract_basic_features
import shutil

import time
from lightgbm import LGBMClassifier
from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from xgboost import XGBClassifier


In [5]:
['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1',  'II', 'aVR', 'V3', 'III', 'V5']

['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1', 'II', 'aVR', 'V3', 'III', 'V5']

In [6]:
feature39_name = [
    "var", "skew", "kurtosis", "perdiogram_1", "perdiogram_2", "perdiogram_3",
    "perdiogram_4", "qrs_areas_mean", "qrs_areas_max", "qrs_areas2_mean",
    "qrs_areas2_max", "qrs_malin_mean", "qrs_malin_max", "qrs_malin2_mean",
    "qrs_malin2_max", "n_plus_mean", "n_plus_max", "n_plus2_mean",
    "n_plus2_max", "v40_mean", "v40_max", "v40_2_mean", "v40_2_max",
    "freq_ratio_1", "freq_ratio_2", "freq_ratio_3", "freq_ratio_4",
    "freq_ratio_5", "lorenz_plot", "rr_var", "rr_var_1", "rr_var_2", "log_rr",
    "log_rr_1_abs", "sample_entropy_1", "sample_entropy_2",
    "shannon_entropy_1", "shannon_entropy_2", "correlation"
]

In [7]:
feature39_name_II = [i+'_II' for i in feature39_name]
feature39_name_V3 = [i+'_V3' for i in feature39_name]
feature39_name_V6 = [i+'_V6' for i in feature39_name]
feature39_name_aVF = [i+'_aVF' for i in feature39_name]

In [8]:
features39_pd = pd.DataFrame(columns=feature39_name_II,index=range(len(train_files)))
#features39_pd.head(2)
if os.path.exists("./feature39/features39_II.csv"):
    features39_pd_II = pd.read_csv("./feature39/features39_II.csv")
else:
    for i,record in tqdm(enumerate(train_files)):
        feat = []
        ecg = sio.loadmat(train_dataset_path+record)
        feat.extend(extract_basic_features(ecg["II"][0],30000)[0])
        features39_pd.loc[i] = feat

    features39_pd.to_csv("./feature39/features39_II.csv",index=False)

In [9]:
features39_pd = pd.DataFrame(columns=feature39_name_V3,index=range(len(train_files)))
#features39_pd.head(2)
if os.path.exists("./feature39/features39_V3.csv"):
    features39_pd_V3 = pd.read_csv("./feature39/features39_V3.csv")
else:
    for i,record in tqdm(enumerate(train_files)):
        feat = []
        ecg = sio.loadmat(train_dataset_path+record)
        feat.extend(extract_basic_features(ecg["V3"][0],30000)[0])
        features39_pd.loc[i] = feat

    features39_pd.to_csv("./feature39/features39_V3.csv",index=False)

In [10]:
features39_pd = pd.DataFrame(columns=feature39_name_aVF,index=range(len(train_files)))
#features39_pd.head(2)
if os.path.exists("./feature39/features39_aVF.csv"):
    features39_pd_aVF = pd.read_csv("./feature39/features39_aVF.csv")
else:
    for i,record in tqdm(enumerate(train_files)):
        feat = []
        ecg = sio.loadmat(train_dataset_path+record)
        feat.extend(extract_basic_features(ecg["aVF"][0],30000)[0])
        features39_pd.loc[i] = feat

    features39_pd.to_csv("./feature39/features39_aVF.csv",index=False)

In [11]:
features39_pd_aVF.tail()

,var_aVF,skew_aVF,kurtosis_aVF,perdiogram_1_aVF,perdiogram_2_aVF,perdiogram_3_aVF,perdiogram_4_aVF,qrs_areas_mean_aVF,qrs_areas_max_aVF,qrs_areas2_mean_aVF,...,rr_var_aVF,rr_var_1_aVF,rr_var_2_aVF,log_rr_aVF,log_rr_1_abs_aVF,sample_entropy_1_aVF,sample_entropy_2_aVF,shannon_entropy_1_aVF,shannon_entropy_2_aVF,correlation_aVF
6495,0.0002,-0.973077,5.619959,-2650.546673,-2726.159403,-3797.366533,-4645.815144,-0.403650,0.291697,-0.370628,...,0.000092,0.000102,0.000157,5.240436,0.417410,1.945910,2.014903,3.106891,3.664498,0.017238
6496,0.0002,0.320714,4.101945,-2650.010695,-2729.089201,-3506.634461,-4685.315704,-0.161361,-0.022024,0.064970,...,0.000022,0.000051,0.000153,3.640313,0.307430,3.806662,0.000000,3.121928,2.947703,0.013937
6497,0.0002,4.478735,27.801425,-2935.066394,-2580.475229,-3422.428275,-4410.786190,0.553006,0.609383,0.748391,...,0.000273,0.000467,0.001062,3.291774,0.624371,2.890372,0.000000,2.947703,3.000000,0.023624
6498,0.0002,2.959382,13.857385,-2869.382688,-2510.926131,-3406.283269,-4391.724005,0.277225,0.469302,0.443753,...,0.000152,0.000203,0.000354,4.931958,0.733631,4.510860,0.000000,3.807355,3.546594,0.010383
6499,0.0002,4.321433,22.709594,-2899.025262,-2507.509796,-3426.970090,-4490.410194,0.779407,0.851157,0.917017,...,0.000086,0.000110,0.000194,3.639155,0.346044,3.113515,0.000000,2.921928,3.169925,0.033133


In [12]:
features39_pd_V3.tail()

,var_V3,skew_V3,kurtosis_V3,perdiogram_1_V3,perdiogram_2_V3,perdiogram_3_V3,perdiogram_4_V3,qrs_areas_mean_V3,qrs_areas_max_V3,qrs_areas2_mean_V3,...,rr_var_V3,rr_var_1_V3,rr_var_2_V3,log_rr_V3,log_rr_1_abs_V3,sample_entropy_1_V3,sample_entropy_2_V3,shannon_entropy_1_V3,shannon_entropy_2_V3,correlation_V3
6495,0.0002,-0.254658,10.132419,-3139.927294,-2857.684944,-3509.610393,-4455.358919,-0.142546,-0.107293,0.016435,...,0.000015,0.000026,0.000076,4.936711,0.306514,2.901422,0.0,3.093069,3.392747,0.001619
6496,0.0002,-1.491958,13.184237,-2805.594486,-2682.732160,-3399.717929,-4391.845184,-0.333410,-0.039578,-0.207275,...,0.000021,0.000051,0.000156,3.640321,0.300236,0.000000,0.0,3.321928,3.169925,0.021378
6497,0.0002,0.469768,9.581992,-2776.026712,-2573.486894,-3448.171587,-4560.492242,0.036918,0.116862,0.274402,...,0.000271,0.000460,0.001034,3.291801,0.625959,2.890372,0.0,3.169925,3.000000,0.009525
6498,0.0002,3.823176,18.814163,-3002.559350,-2566.866777,-3365.073230,-4329.617643,0.483573,0.653279,0.638949,...,0.000151,0.000198,0.000341,4.932016,0.729409,0.000000,0.0,3.807355,3.700440,0.005273
6499,0.0002,-1.112012,10.492450,-2931.015841,-2495.384526,-3401.777985,-4502.923833,-0.301827,-0.183863,-0.144460,...,0.000082,0.000101,0.000174,3.639223,0.333082,2.708050,0.0,3.321928,2.947703,0.019555


In [13]:
features39_pd_II.tail()

,var_II,skew_II,kurtosis_II,perdiogram_1_II,perdiogram_2_II,perdiogram_3_II,perdiogram_4_II,qrs_areas_mean_II,qrs_areas_max_II,qrs_areas2_mean_II,...,rr_var_II,rr_var_1_II,rr_var_2_II,log_rr_II,log_rr_1_abs_II,sample_entropy_1_II,sample_entropy_2_II,shannon_entropy_1_II,shannon_entropy_2_II,correlation_II
6495,0.0002,0.841733,1.038107,-2669.878220,-2771.567454,-3810.314383,-4793.906284,-0.196164,0.078146,-0.032934,...,0.000016,0.000029,0.000076,4.936672,0.291623,2.901422,0.0,3.093069,3.546594,0.016704
6496,0.0002,0.519052,4.918307,-2685.042392,-2772.809633,-3488.640617,-4599.295090,-0.135883,-0.085005,0.096052,...,0.000021,0.000052,0.000160,3.640329,0.303798,3.113515,0.0,2.921928,2.947703,0.013599
6497,0.0002,3.864339,22.254192,-2888.383063,-2552.276433,-3438.568042,-4487.089356,0.576914,0.632430,0.785903,...,0.000269,0.000458,0.001032,3.291834,0.620898,2.484907,0.0,3.169925,3.000000,0.023461
6498,0.0002,3.628166,16.663626,-2909.116812,-2470.863495,-3406.564221,-4413.050057,0.509271,0.647335,0.646033,...,0.000140,0.000185,0.000324,4.932390,0.702343,2.901422,0.0,3.807355,3.392747,0.007635
6499,0.0002,3.663893,17.212268,-2886.312087,-2515.476059,-3434.087598,-4546.008119,0.725843,0.782350,0.872075,...,0.000088,0.000114,0.000206,3.639105,0.343690,2.708050,0.0,2.921928,3.169925,0.026671


In [14]:
labels = pd.read_csv("reference.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0003,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0004,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
#!/bin/python
from ctypes import  *
import os

class StructPointer(Structure):  
    _fields_ = [("RR_mean", c_double),("RR_min", c_double),("RR_max", c_double),("HR_median", c_double),
    ("AFEv", c_int), ("IrrEv", c_int),("CV", c_double), ("MAD",c_double), ("CosEn", c_double),("Radius", c_double),
    ("CV_deltaRR", c_double)]  
	
hr=[240,200,256,260]  #,248,238,250,239,243,270
carrary = (c_int * len(hr))(*hr)
fs=256
mylib = cdll.LoadLibrary("featcaculate.so")
mylib.AFDet.restype = POINTER(StructPointer) 
c_result = mylib.AFDet(carrary,len(hr),fs,1)

# print(c_result.contents.AFEv)
# print(c_result.contents.CV)
print("test finished!")

test finished!


In [122]:
os.getcwd()

'/media/jdcloud'

In [121]:
c_result.contents.CV

0.09927874910213115

In [15]:
import lightgbm as lgb
lgb_params2={
 'learning_rate' : 0.01,
 'n_estimators':819,
 'max_depth':8,
 'num_leaves':115,
 'min_child_weight':0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'min_child_samples':21,
 'objective':'multiclass',
 'reg_alpha':0.15,
 'reg_lambda' : 0.01,
 'num_class': 9,
 'n_jobs':4,
 #class_weight ="1",
 'random_state' :27}

In [16]:
train_df = pd.concat([features39_pd_II,features39_pd_V3,features39_pd_aVF],axis=1)#features315_pd#
feature_name = feature39_name_II+feature39_name_V3+feature39_name_aVF#feature39_name
train_labels = labels['label1'].values

In [25]:
from sklearn.preprocessing import LabelEncoder

In [28]:
label_encoder = LabelEncoder()
label_encoder = LabelEncoder().fit_transform(train_labels)
#label_encoder_y = LabelEncoder().(train_labels)

In [36]:
def preprocess_y(y,num_class=9):
    bin_label = np.zeros((y.shape[0],num_class))
    for i in range(y.shape[0]):
        label_nona = labels.loc[y.index.tolist()[i]].dropna()
        for j in range(1,label_nona.shape[0]):
            bin_label[i,int(label_nona[j])]=1
    return bin_label

In [38]:
bin_labels = preprocess_y(labels)

In [33]:
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer

In [107]:
OneVsRestClassifier(xgb.XGBClassifier(learning_rate=0.005,n_eatimators=500))

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.005, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_eatimators=500,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1),
          n_jobs=None)

In [116]:
n_folds = 5
n_classes = 9
kfold = KFold(n_folds,shuffle=True,random_state=123)

#kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=123)
kf = kfold.split(train_data_df, train_label)

blend_train = np.zeros((train_df.shape[0],n_classes))
blend_test = np.zeros((val_df.shape[0],n_folds,n_classes))

for i,(train_ind,val_ind) in enumerate(kf):
    t = time.time()
    print("fold : ",i+1)
    X = train_df[feature_name].astype('float32')
    X_train = X.iloc[train_ind]
    y_train = bin_labels[train_ind]
    
    X_val = X.iloc[val_ind]
    y_val = bin_labels[val_ind]
    
    X_test = val_df[feature_name].astype('float32')
    
    clf = OneVsRestClassifier(xgb.XGBClassifier(learning_rate=0.005,n_eatimators=500))
    clf.fit(X_train,y_train)
    
    y_p_v = clf.predict_proba(X_val)
    y_p_test = clf.predict_proba(X_test)
    
    blend_train[val_ind,:] = y_p_v
    blend_test[:,i,:] = y_p_test
    
    f1 = f1_score(y_val,1*y_p_v>0.4,average='macro')
    print("... time passed: {0:.1f}sec , fold F1 : {1:.4f}".format(time.time()-t,f1))
    
f1 = f1_score(bin_labels,1*blend_train>0.4,average='macro')
    
    

fold :  1
... time passed: 29.0sec , fold F1 : 0.5603
fold :  2
... time passed: 29.2sec , fold F1 : 0.5650
fold :  3
... time passed: 29.1sec , fold F1 : 0.5613
fold :  4
... time passed: 29.1sec , fold F1 : 0.5342
fold :  5
... time passed: 29.4sec , fold F1 : 0.5690


In [114]:
cv_pred_all = 0
en_amount = 1

import lightgbm as lgb
lgb_params={
 'learning_rate' : 0.01,
 'n_estimators':819,
 'max_depth':8,
 'num_leaves':115,
 'min_child_weight':0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'min_child_samples':21,
 'objective':'binary',#'multiclass',
 'reg_alpha':0.15,
 'reg_lambda' : 0.01,
 'num_class': 1,
 'n_jobs':4,
  'verbosity':2,
 #class_weight ="1",
 'random_state' :27}

for seed in range(en_amount):
    print("************************")
    NFOLDS = 5
    train_label = train_labels#train_data['score']
    train_data_df = train_df[feature_name].astype('float32') #feature_name  columns
    kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
    kf = kfold.split(train_data_df, train_label)

    #train_data_use = train_data.drop(['uid','score','blk_list_flag'], axis=1)
    #test_data_use = test_data.drop(['uid','blk_list_flag'], axis=1)
    train_data_use = train_data_df
    # cv_pred = np.zeros(test_data.shape[0])
    valid_best_l2_all = 0
    oof = np.zeros(train_data_df.shape[0])
    feature_importance_df = pd.DataFrame()
    count = 0
    for i, (train_fold, validate) in enumerate(kf):
        print('fold: ',i, ' training')

        X = train_df[feature_name].astype('float32')
        X_train = X.iloc[train_fold]
        y_train = bin_labels[train_fold]

        X_val = X.iloc[validate]
        y_val = bin_labels[validate]

        X_test = val_df[feature_name].astype('float32')
        #print(X_validate)
        #break
        #dtrain = lgb.Dataset(X_train, label_train)
        #dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)
        bst = OneVsRestClassifier(lgb.LGBMClassifier(**lgb_params))
        #, dtrain, num_boost_round=819, valid_sets=dvalid, verbose_eval=200,early_stopping_rounds=500)
        bst.fit(X_train, y_train)
        
        y_p_v = bst.predict_proba(X_val)
        y_p_test = bst.predict_proba(X_test)

        blend_train[validate,:] = y_p_v
        blend_test[:,i,:] = y_p_test

        f1 = f1_score(y_val,1*y_p_v>0.4,average='macro')
        print("... time passed: {0:.1f}sec , fold F1 : {1:.4f}".format(time.time()-t,f1))
    
        #cv_pred += bst.predict(test_data_use, num_iteration=bst.best_iteration)
        #valid_best_l2_all += bst.best_score['valid_0']['l1']

        count += 1
        
fold_f1_error = f1_score(bin_labels,1*blend_train>0.4,average='macro')
print('fold f1 score is {0}'.format(fold_f1_error))

#F1n,F1a,F1o,F1p,F1 = cinc_f1_score(np.array(oof),np.array(train_labels.values))


************************
fold:  0  training
... time passed: 2915.9sec , fold F1 : 0.6483
fold:  1  training
... time passed: 2967.4sec , fold F1 : 0.6617
fold:  2  training
... time passed: 3020.2sec , fold F1 : 0.6531
fold:  3  training
... time passed: 3068.4sec , fold F1 : 0.6409
fold:  4  training
... time passed: 3117.1sec , fold F1 : 0.6216
fold f1 score is 0.6454172267221155


In [115]:
y_p_tst = blend_test.mean(axis=1)#>0.4

classes = [0,1,2,3,4,5,6,7,8]
thred = 0.4 #0.15
test_y = y_p_tst
y_pred = [[1 if test_y[i,j] >= thred else 0 for j in range(test_y.shape[1])] 
          for i in range(len(test_y))]
pred=[]
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])

with open('answers1.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [17]:
cv_pred_all = 0
en_amount = 1
for seed in range(en_amount):
    print("************************")
    NFOLDS = 10
    train_label = train_labels#train_data['score']
    train_data_df = train_df[feature_name].astype('float32') #feature_name  columns
    kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
    kf = kfold.split(train_data_df, train_label)

    #train_data_use = train_data.drop(['uid','score','blk_list_flag'], axis=1)
    #test_data_use = test_data.drop(['uid','blk_list_flag'], axis=1)
    train_data_use = train_data_df
    # cv_pred = np.zeros(test_data.shape[0])
    valid_best_l2_all = 0
    oof = np.zeros(train_data_df.shape[0])
    feature_importance_df = pd.DataFrame()
    count = 0
    for i, (train_fold, validate) in enumerate(kf):
        print('fold: ',i, ' training')
        X_train, X_validate, label_train, label_validate = \
        train_data_use.iloc[train_fold, :], train_data_use.iloc[validate, :], \
        train_label[train_fold], train_label[validate]
        #print(X_validate)
        #break
        dtrain = lgb.Dataset(X_train, label_train)
        dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)
        bst = lgb.train(lgb_params2, dtrain, num_boost_round=819, valid_sets=dvalid, verbose_eval=200,early_stopping_rounds=500)
        #cv_pred += bst.predict(test_data_use, num_iteration=bst.best_iteration)
        #valid_best_l2_all += bst.best_score['valid_0']['l1']
        k_pred = bst.predict(X_validate, num_iteration=bst.best_iteration)
        oof[validate] = np.argmax(k_pred,axis=1)
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = list(X_train.columns)
        fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
        fold_importance_df["fold"] = count + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        count += 1

oof = list(map(int, oof))
fold_f1_error = f1_score(train_labels, oof, average='macro')
print('fold f1 score is {0}'.format(fold_f1_error))

#F1n,F1a,F1o,F1p,F1 = cinc_f1_score(np.array(oof),np.array(train_labels.values))


************************
fold:  0  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's multi_logloss: 0.95781
[400]	valid_0's multi_logloss: 0.790339
[600]	valid_0's multi_logloss: 0.732317
[800]	valid_0's multi_logloss: 0.714109
Did not meet early stopping. Best iteration is:
[817]	valid_0's multi_logloss: 0.713529
fold:  1  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's multi_logloss: 1.00614
[400]	valid_0's multi_logloss: 0.840589
[600]	valid_0's multi_logloss: 0.787958
[800]	valid_0's multi_logloss: 0.772766
Did not meet early stopping. Best iteration is:
[816]	valid_0's multi_logloss: 0.772532
fold:  2  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's multi_logloss: 1.01114
[400]	valid_0's multi_logloss: 0.867016
[600]	valid_0's multi_logloss: 0.822438
[800]	valid_0's multi_logloss: 0.813291
Did not meet early stopping. Best iteration is:
[810]	valid_0's multi_loglos

In [19]:
val_features39_II = pd.DataFrame(columns=feature39_name_II,index=range(len(val_files)))
for i,record in tqdm(enumerate(val_files)):
    feat = []
    ecg = sio.loadmat(val_dataset_path+record)
    feat.extend(extract_basic_features(ecg["II"][0],30000)[0])
    val_features39_II.loc[i] = feat

500it [00:07, 68.12it/s]


In [20]:
val_features39_V3 = pd.DataFrame(columns=feature39_name_V3,index=range(len(val_files)))
for i,record in tqdm(enumerate(val_files)):
    feat = []
    ecg = sio.loadmat(val_dataset_path+record)
    feat.extend(extract_basic_features(ecg["V3"][0],30000)[0])
    val_features39_V3.loc[i] = feat

500it [00:07, 69.94it/s]


In [21]:
val_features39_aVF = pd.DataFrame(columns=feature39_name_aVF,index=range(len(val_files)))
for i,record in tqdm(enumerate(val_files)):
    feat = []
    ecg = sio.loadmat(val_dataset_path+record)
    feat.extend(extract_basic_features(ecg["aVF"][0],30000)[0])
    val_features39_aVF.loc[i] = feat

500it [00:07, 68.08it/s]


In [22]:
val_df = pd.concat([val_features39_II,val_features39_V3,val_features39_aVF],axis=1)#features315_pd#
feature_name = feature39_name_II+feature39_name_V3+feature39_name_aVF#feature39_name


In [24]:
train_labels

array([8, 8, 8, ..., 0, 0, 0])

In [ ]:
train_dfeatures = train_df[feature_name].values
x_tr_y = bst.predict(train_dfeatures)

#x_tr_y = model.predict(train_x)#X_tr

threshold = np.arange(0.1,0.9,0.1)

out = x_tr_y
y_test = bin_label#y_tr

acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        #acc.append( matthews_corrcoef(y_test[:,i],y_pred))
        acc.append(f1_score(y_test[:,i],y_pred,average='macro'))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []
    
print("best_threshold: ",best_threshold)

y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])

y_pred 

y_test

#best_threshold:  [0.7 0.4 0.5 0.4 0.3 0.2 0.3 0.4 0.4]
#0.022393162393162393

#best_threshold:  [0.7 0.4 0.5 0.4 0.4 0.2 0.4 0.4 0.5]
#0.022615384615384617

hamming_loss(y_test,y_pred)

In [97]:
import csv

dfeatures = val_df[feature_name].values
kpred = bst.predict(dfeatures)

pred = []
thred = 0.15 #0.15
for i in range(kpred.shape[0]):
    pred.append( list(np.hstack(np.argwhere(kpred[i]>thred))))
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

### First 315 features 

In [9]:
from features_st import entropy,pyeeg
from features_st.features.features_submit import *

In [10]:
# Instantiate Features object
#file_name = "TRAIN0001"
def generate_features315(dataset_path,file_name):
    ecg_features = FeaturesSubmit(
        file_path=os.path.join(dataset_path, file_name+'.mat'),
        file_name=file_name,
        fs=500,
        feature_groups=[
            'full_waveform_statistics',
            'heart_rate_variability_statistics',
            'template_statistics'
        ]
    )

    # Calculate ECG features
    ecg_features.calculate_features(
        filter_bandwidth=[3, 45], show=False,
        normalize=True, polarity_check=True,
        template_before=0.25, template_after=0.4
    )

    # Get features DataFrame
    features = ecg_features.get_features()
    
    return features

In [172]:
if os.path.exists("features315_II.csv"):
    features315_pd = pd.read_csv("features315_II.csv")
else:
    features315_pd = generate_features315(train_dataset_path,"TRAIN0001")
    for file_name in tqdm(train_files[1:]):
        df_temp = generate_features315(train_dataset_path,file_name.strip('.mat'))
        features315_pd = pd.concat([features315_pd,df_temp])
    features315_pd.to_csv("features315_II.csv",index=False)

In [27]:
features315_name = set(features315_pd.columns.values) - set(['File_name'])
len(features315_name)

314

In [29]:
import sklearn.preprocessing as sp

In [79]:
labels = pd.read_csv("reference.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0003,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0004,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
ex_labels = labels.copy()
ex_features315_pd = features315_pd.copy()

In [107]:
for i in range(labels.shape[0])[:]:
    label_nona = labels.loc[i].dropna()
    if label_nona.shape[0]>2:
        for j in range(1,label_nona.shape[0]-1):
            #print(label_nona['File_name'])
            #print(label_nona['label'+str(j+1)])
            label_row = [label_nona['File_name'],label_nona['label'+str(j+1)],'','','','','','','']
            ex_labels.loc[ex_labels.shape[0]]=label_row
            ex_features315_pd.loc[ex_labels.shape[0]] = ex_features315_pd.iloc[i].values

In [111]:
ex_labels.to_csv("ex_labels.csv",index=False)

In [110]:
ex_features315_pd.to_csv("ex_features315_II.csv",index=False)

In [97]:
labels["label1"].value_counts()

0    1953
8    1547
3     714
6     571
5     520
2     449
1     430
7     205
4     111
Name: label1, dtype: int64

In [65]:
sum1_label=0
sum2_label=0
sum3_label=0
sum4_label=0
sum5_label=0
sum6_label=0
sum7_label=0
bin_label = np.zeros((6500,9))
for i in range(labels.shape[0]):
    label_nona = labels.loc[i].dropna()
    if label_nona.shape[0] == 2:
        sum1_label+=1
    if label_nona.shape[0] == 3:
        sum2_label+=1
    if label_nona.shape[0] == 4:
        sum3_label+=1
    if label_nona.shape[0] == 5:
        sum4_label+=1
    if label_nona.shape[0] == 6:
        sum5_label+=1
    if label_nona.shape[0] == 7:
        sum6_label+=1
    if label_nona.shape[0] == 8:
        sum7_label+=1
    for j in range(1,label_nona.shape[0]):
        bin_label[i,int(label_nona[j])]=1

In [71]:
sum6_label

1

In [94]:
sum1_label + sum2_label*2 + sum3_label*3 + sum4_label*4 + sum5_label*5 + sum6_label*6 + sum7_label*7

7703

In [112]:
import lightgbm as lgb
lgb_params2={
 'learning_rate' : 0.01,
 'n_estimators':819,
 'max_depth':8,
 'num_leaves':115,
 'min_child_weight':0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'min_child_samples':21,
 'objective':'multiclass',
 'reg_alpha':0.15,
 'reg_lambda' : 0.01,
 'num_class': 9,
 'n_jobs':4,
 #class_weight ="1",
 'random_state' :27}

In [173]:
train_df = features315_pd#ex_features315_pd#features315_pd#features39_pd
feature_name = ['t_wave_eng',
 's_wave_amp',
 'p_wave_eng',
 'q_wave_amp',
 't_wave_corr_coeff_std',
 't_wave_amp',
 'pt_time',
 'p_wave_time',
 'rri_skew',
 'p_wave_amp',
 'p_wave_corr_coeff_std',
 'rri_multiscale_entropy',
 'rpeak_multiscale_entropy',
 'pq_time',
 't_wave_sample_entropy_median',
 'swt_a_1_med_power_ratio',
 'diff_rri_multiscale_entropy',
 'full_waveform_median',
 't_wave_corr_coeff_median',
 'p_wave_permutation_entropy_std',
 'diff2_rri_multiscale_entropy',
 't_wave_higuchi_fractal_median',
 'diff_rri_skew',
 'template_swt_a_4_low_power_ratio',
 't_wave_permutation_entropy_std',
 'diff_rri_mean',
 'full_waveform_mean',
 't_wave_approximate_entropy_median',
 'rri_p3_pearson_p_value',
 'p_wave_higuchi_fractal_median',
 'template_swt_d_1_med_power_ratio',
 't_wave_time',
 's_wave_time',
 'rri_min',
 't_wave_hurst_exponent_std',
 'full_waveform_skew',
 'swt_a_1_higuchi_fractal',
 'heart_rate_multiscale_entropy',
 'p_wave_multiscale_entropy_std',
 'diff_rri_fisher_info',
 'p_wave_corr_coeff_median',
 'pt_time_std',
 'full_waveform_min',
 'rri_very_high_frequency_power',
 'qrs_corr_coeff_std',
 'qt_time',
 't_wave_approximate_entropy_std',
 'template_swt_a_1_low_power_ratio',
 'rri_kurtosis',
 'p_wave_sample_entropy_median',
 'full_waveform_max',
 'diff2_rri_svd_entropy',
 'template_swt_d_2_med_power_ratio',
 't_wave_sample_entropy_std',
 't_wave_higuchi_fractal_std',
 'qrs_energy_pearson_p_value',
 'p_wave_amp_std',
 'swt_d_1_med_power_ratio',
 'p_wave_approximate_entropy_std',
 'rpeaks_bad',
 't_wave_multiscale_permutation_entropy_median',
 'qsi_pearson_coeff',
 'template_corr_coeff_std',
 'st_time',
 'template_swt_a_1_med_power_ratio',
 'swt_a_1_low_power_ratio',
 'q_wave_amp_std',
 'p_wave_time_std',
 'full_waveform_std',
 'rpeak_kurtosis',
 'swt_a_1_high_power_ratio',
 'rri_cluster_ssd_3',
 'template_swt_a_1_high_power_ratio',
 'diff2_rri_max',
 'heart_rate_skew',
 't_wave_amp_std',
 'full_waveform_kurtosis',
 'diff_rri_svd_entropy',
 'swt_d_3_high_power_ratio',
 'diff2_rri_min',
 'heart_rate_kurtosis',
 'swt_d_4_higuchi_fractal',
 'pq_time_std',
 'rri_p3_pearson_coeff',
 'p_wave_higuchi_fractal_std',
 'diff2_rri_mean',
 'template_swt_d_3_high_power_ratio',
 'p_wave_approximate_entropy_median',
 'fragmentation_ials',
 'diff2_rri_morbidity',
 'rri_p2_pearson_p_value',
 'p_wave_hurst_exponent_std',
 't_wave_hurst_exponent_median',
 'diff2_rri_kurtosis',
 'template_swt_d_1_low_power_ratio',
 'diff2_rri_fisher_info',
 'diff_rri_kurtosis',
 'rpeak_skew',
 'rri_spectral_entropy',
 'qrs_energy_std'][:120]#
feature_name = features315_name#feature39_name

In [169]:
train_labels = labels["label1"].values#labels["label1"].values  #bin_label
len(train_labels)

6500

In [174]:
train_df.shape

(6500, 315)

In [ ]:
cv_pred_all = 0
en_amount = 1
for seed in range(en_amount):
    print("************************")
    NFOLDS = 10
    train_label = train_labels#train_data['score']
    train_data_df = train_df[feature_name].astype('float32') #feature_name  columns
    kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
    kf = kfold.split(train_data_df, train_label)

    #train_data_use = train_data.drop(['uid','score','blk_list_flag'], axis=1)
    #test_data_use = test_data.drop(['uid','blk_list_flag'], axis=1)
    train_data_use = train_data_df
    # cv_pred = np.zeros(test_data.shape[0])
    valid_best_l2_all = 0
    oof = np.zeros(train_data_df.shape[0])
    feature_importance_df = pd.DataFrame()
    count = 0
    for i, (train_fold, validate) in enumerate(kf):
        print('fold: ',i, ' training')
        X_train, X_validate, label_train, label_validate = \
        train_data_use.iloc[train_fold, :], train_data_use.iloc[validate, :], \
        train_label[train_fold], train_label[validate]
        #print(X_validate)
        #break
        dtrain = lgb.Dataset(X_train, label_train)
        dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)
        bst = lgb.train(lgb_params2, dtrain, num_boost_round=819, valid_sets=dvalid, verbose_eval=200,early_stopping_rounds=500)
        #cv_pred += bst.predict(test_data_use, num_iteration=bst.best_iteration)
        #valid_best_l2_all += bst.best_score['valid_0']['l1']
        k_pred = bst.predict(X_validate, num_iteration=bst.best_iteration)
        oof[validate] = np.argmax(k_pred,axis=1)
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = list(X_train.columns)
        fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
        fold_importance_df["fold"] = count + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        count += 1

oof = list(map(int, oof))
fold_f1_error = f1_score(train_labels, oof, average='macro')
print('fold f1 score is {0}'.format(fold_f1_error))

#F1n,F1a,F1o,F1p,F1 = cinc_f1_score(np.array(oof),np.array(train_labels.values))


************************
fold:  0  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's multi_logloss: 0.809684
[400]	valid_0's multi_logloss: 0.655225


In [121]:
feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:100].index.tolist()

['t_wave_eng',
 'pt_time',
 'p_wave_time',
 'p_wave_eng',
 's_wave_amp',
 't_wave_amp',
 'rri_skew',
 'p_wave_corr_coeff_median',
 't_wave_higuchi_fractal_median',
 't_wave_sample_entropy_median',
 'p_wave_amp',
 'rri_multiscale_entropy',
 't_wave_corr_coeff_std',
 'full_waveform_median',
 'q_wave_amp',
 'p_wave_corr_coeff_std',
 't_wave_time',
 'swt_a_1_med_power_ratio',
 't_wave_corr_coeff_median',
 'template_swt_a_4_low_power_ratio',
 'full_waveform_skew',
 't_wave_approximate_entropy_median',
 'st_time',
 'qt_time',
 'pq_time',
 'template_swt_d_1_med_power_ratio',
 'rri_cluster_ssd_3',
 'rri_kurtosis',
 's_wave_time',
 'full_waveform_min',
 'p_wave_higuchi_fractal_median',
 'rpeaks_bad',
 'rpeak_multiscale_entropy',
 'diff_rri_skew',
 'qrs_corr_coeff_std',
 'diff_rri_multiscale_entropy',
 'rri_min',
 'diff_rri_mean',
 't_wave_permutation_entropy_std',
 'swt_d_1_med_power_ratio',
 'full_waveform_max',
 'pt_time_std',
 'template_swt_a_1_low_power_ratio',
 'full_waveform_kurtosis',
 '

In [55]:
oof = list(map(int, oof))
fold_f1_error = f1_score(train_labels, oof, average='macro')
print('fold f1 score is {0}'.format(fold_f1_error))

fold f1 score is 0.7022919519907358


In [56]:
bst.save_model("lightgbm_05141_temp315.bin")

In [57]:
import lightgbm as lgb 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

bst = lgb.Booster(model_file='lightgbm_05141_temp315.bin')  #lgb.bin  lightgbm_0327

In [58]:
if os.path.exists("val_features315_II.csv"):
    pass
else:
    val_features315_pd = generate_features315(val_dataset_path,"VAL001")
    for file_name in tqdm(val_files[1:]):
        df_temp = generate_features315(val_dataset_path,file_name.strip('.mat'))
        val_features315_pd = pd.concat([val_features315_pd,df_temp])
    val_features315_pd.to_csv("val_features315_II.csv",index=False)

In [59]:
import csv

dfeatures = val_features315_pd[feature_name].values
kpred = bst.predict(dfeatures)

pred = []
for i in range(kpred.shape[0]):
    pred.append( list(np.hstack(np.argwhere(kpred[i]>0.15))))
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [143]:
val_features39_pd = pd.DataFrame(columns=feature39_name,index=range(len(val_files)))
for i,record in tqdm(enumerate(val_files)):
    feat = []
    ecg = sio.loadmat(val_dataset_path+record)
    feat.extend(extract_basic_features(ecg["II"][0],30000)[0])
    val_features39_pd.loc[i] = feat
    
val_features39_pd.to_csv("val_features39.csv",index=False)

500it [00:09, 52.68it/s]


In [145]:
val_features39_pd.tail()

,var,skew,kurtosis,perdiogram_1,perdiogram_2,perdiogram_3,perdiogram_4,qrs_areas_mean,qrs_areas_max,qrs_areas2_mean,...,rr_var,rr_var_1,rr_var_2,log_rr,log_rr_1_abs,sample_entropy_1,sample_entropy_2,shannon_entropy_1,shannon_entropy_2,correlation
495,6.66667e-05,4.30897,24.2626,-9299.66,-8125.87,-11318.8,-14838.5,0.43602,0.722651,0.471016,...,0.00179353,0.00472905,0.0146254,9.49078,3.87286,1.62161,1.39666,4.8125,4.69613,0.00856784
496,0.000133333,4.16013,20.413,-4555.15,-3908.04,-5267.6,-7132.83,0.625227,0.656808,0.709537,...,7.18617e-05,1.62732e-05,2.65265e-05,6.12745,0.327509,2.951,0,3.79465,3.57247,0.0198841
497,0.0002,1.88521,12.6546,-2614.96,-2569.6,-4061.58,-5287.63,0.186569,2.72928,0.177976,...,0.00938753,0.0221122,0.0698939,5.11229,2.99427,2.84158,1.25276,3.75,3.90689,0.0212713
498,0.0002,3.64245,16.1741,-3055.34,-2523.26,-3398.77,-4542.8,0.675001,0.706521,0.792839,...,8.2744e-05,3.00286e-05,8.33071e-05,4.62154,0.315956,2.74727,1.38629,3.54659,3.58496,0.00334011
499,0.000133333,2.58085,10.8808,-4383.81,-4012.28,-5184.31,-7209.27,0.430938,0.504038,0.571314,...,5.08705e-05,8.5478e-05,0.000133537,6.41416,0.702254,2.94444,2.19722,3.93214,3.9477,0.00822882


In [264]:
dfeatures = val_features39_pd[feature39_name].values
kpred = bst.predict(dfeatures)

In [265]:
pred = []
for i in range(kpred.shape[0]):
    pred.append( list(np.hstack(np.argwhere(kpred[i]>0.15))))

In [266]:
labels.columns.tolist()

['File_name',
 'label1',
 'label2',
 'label3',
 'label4',
 'label5',
 'label6',
 'label7',
 'label8']

In [267]:

import random
random.randint(0,8)

5

In [268]:
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [4]:
import numpy as np
import pandas as pd

ecg12_data=[]
for k,v in ecg.items():
    if k in ["__header__","__globals__","__version__"]:
        continue
    elif k in ["sex","age"]:
        continue
    else:
        ecg12_data.append(v[0])
ecg = np.array(ecg12_data)